In [0]:
from pyspark.sql.functions import explode, split, trim, col, lower
class streamWordCount:
    def __init__(self):
        self.base_data = "/Volumes/spark-catalog/sandbox/sample_data/"
    
    def getRawData(self):
        lines = (spark.readStream
                     .format("text") \
                     .option("lineSep", ".") \
                     .load(f"{self.base_data}/text_data_*.txt"))
        return lines.select(explode(split(col("value"), " ")).alias("word"))
    
    def getRefinedData(self, rawData):
        return (self.getRawData()
                    .select(trim(lower(col("word"))).alias("word"))
                    .where(col("word") != "")
                    .where("word rlike '[a-z]'"))
    
    def getwordCount(self,refinedData):
        return refinedData.groupBy("word").count()

    def overwriteWordCount(self,wordCount):
        wordCount.writeStream \
                 .format("delta") \
                 .option("checkpointLocation", f"{self.base_data}/checkpoint/word_count") \
                 .outputMode("complete") \
                 .toTable("word_count_stream")
    
    def streamRun(self):
        print("Starting Streaming Word Count")
        rawData = self.getRawData()
        refinedData = self.getRefinedData(rawData)
        wordCount = self.getwordCount(refinedData)
        query = self.overwriteWordCount(wordCount)
        print("DONE")
        return(query)

